In [2]:
import dotenv
dotenv.load_dotenv()

True

# Language Model input and output

<p><strong>To understand how input and output handling in LangChain works it is beneficial to first understand what format the model expects the input to be and what it outputs</strong></p>

<p>Under the hood LLM takes a string input and produces a string output.
But for various models the input format may differ depending on how the training data was formatted.
</p>

<p>OpenAI API offers two endpoints. ChatCompletion and Completion, the second one will be depreciated on January 04, 2024 but lets compare them to see and understand the difference and how LangChain helps as a high level wrapper around them. 
Completion. </p>

<p>For the Completion API the input is left unhanged, the same as we write it. The call to the API with their python library looks like that:
</p>

In [3]:
import openai

behaviour = "an expert at telling jokes"
topic = "bear"

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=f"""\
You are {behaviour}.
User question: Tell me a joke about {topic}.""",
    temperature=0.0,
)

print(response)
# read more: https://platform.openai.com/docs/guides/gpt/completions-api

{
  "id": "cmpl-86cZ5euhXwVDrZ0xC6wDtYTXYOhsb",
  "object": "text_completion",
  "created": 1696588903,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "\n\nWhy did the bear refuse to wear shoes?\n\nBecause he had bear feet!",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 18,
    "completion_tokens": 16,
    "total_tokens": 34
  }
}


<p>As we can see the response is a json object and to extract the generated text we have to write </p>

In [4]:
response['choices'][0]['text']

'\n\nWhy did the bear refuse to wear shoes?\n\nBecause he had bear feet!'

<p>For the ChatCompletion API, the input consist of a list of messages which represent current chat history and may also include a system message which helps set the behavior of the assistant.</p>

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": f"You are {behaviour}."},
        {"role": "user", "content": "Hi. What's your name?"},
        {"role": "assistant", "content": f"Hello. I am {behaviour}. What can I do for you today?"},
        {"role": "user", "content": f"Tell me a joke about {topic}"}
    ],
    temperature=0.0,
)

print(response)

{
  "id": "chatcmpl-86cZ6hMirsiNCEheLF37szPSAQXFa",
  "object": "chat.completion",
  "created": 1696588904,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Sure, here's a bear-themed joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 58,
    "completion_tokens": 24,
    "total_tokens": 82
  }
}


<p>The difference comes from the fact that chat modes are trained with specifically formatted input. While we don't know the format for OpenAI models, lets see for example format for open source Meta Lama2:</p>

In [6]:
print('[INST] <<SYS>>\nSystem_Message_Here\n<</SYS>>\n\nUser_Msg_1 [/INST] Assistant_Msg_1 [INST] User_Msg_2 [/INST] Assistant_Msg_2 [INST] User_Msg_3 [/INST]')

[INST] <<SYS>>
System_Message_Here
<</SYS>>

User_Msg_1 [/INST] Assistant_Msg_1 [INST] User_Msg_2 [/INST] Assistant_Msg_2 [INST] User_Msg_3 [/INST]


### How LangChain can help?

<p>When building more complex applications one don't want to focus on parsing json or constantly checking for API specification changes from different providers.

Here comes LangChain which removes the burden of doing it on one's own.  
It provides an abstraction for working with language models, along with implementation for most popular providers. </p>

<p>Let's see how the above examples can be implemented in it.

Note that for prompt we don't use python f-string</p>

In [7]:
from langchain.chat_models import ChatOpenAI # for ChatCompletion API
from langchain.llms import OpenAI # for Completion API
from langchain.prompts import (
    PromptTemplate, 
    ChatPromptTemplate, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.prompts.chat import HumanMessage

completion_prompt = PromptTemplate(
    template="""\
You are {behaviour}.
User question: Tell me a joke about {topic}.""",
    input_variables=['behaviour', 'topic'])

completion_api = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.0,)

completion_resp = completion_api(completion_prompt.format(behaviour=behaviour, topic=topic))


chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are {behaviour}."),
        HumanMessage(content="Hi. What's your name?"),
        AIMessagePromptTemplate.from_template("Hello. I am {behaviour}. What can I do for you today?"),
    ]
    # prompts support addition
) + HumanMessagePromptTemplate.from_template("Tell me a joke about {topic}")

chat_api = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0)

chat_resp = chat_api(chat_prompt.format_messages(behaviour=behaviour, topic=topic))

completion_resp, chat_resp

('\n\nWhy did the bear refuse to wear shoes?\n\nBecause he had bear feet!',
 AIMessage(content="Sure, here's a bear-themed joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"))

<p>Under the hood ChatOpenAI creates the desired list of dicts with correct format for us, while OpenAI class expects only input string. We can inspect the formats to check how they look like:</p>

In [8]:
completion_prompt.format(behaviour=behaviour, topic=topic)

'You are an expert at telling jokes.\nUser question: Tell me a joke about bear.'

In [9]:
chat_prompt.format_messages(behaviour=behaviour, topic=topic)

[SystemMessage(content='You are an expert at telling jokes.'),
 HumanMessage(content="Hi. What's your name?"),
 AIMessage(content='Hello. I am an expert at telling jokes. What can I do for you today?'),
 HumanMessage(content='Tell me a joke about bear')]

In [10]:
chat_api._create_message_dicts(chat_prompt.format_messages(behaviour=behaviour, topic=topic), stop=None)[0]

[{'role': 'system', 'content': 'You are an expert at telling jokes.'},
 {'role': 'user', 'content': "Hi. What's your name?"},
 {'role': 'assistant',
  'content': 'Hello. I am an expert at telling jokes. What can I do for you today?'},
 {'role': 'user', 'content': 'Tell me a joke about bear'}]

#### Change provider

If you want to change LLM provider, LangChain makes it very easy.
For example, let's say you want to test your app with Anthropic:
The only thing you have to do is to hange the model, templates stay the same.

In [11]:
from langchain.chat_models import ChatAnthropic
import os
# this must be set to instantiate the class
os.environ['ANTHROPIC_API_KEY'] = 'dummy key'
print(ChatAnthropic()._convert_messages_to_prompt(chat_prompt.format_messages(behaviour=behaviour, topic=topic)))



Human: <admin>You are an expert at telling jokes.</admin>

Human: Hi. What's your name?

Assistant: Hello. I am an expert at telling jokes. What can I do for you today?

Human: Tell me a joke about bear

Assistant:
